### Topic Summary


In this project I aim to develop a generalizable novel methodology for identifying website list in a target category. And I will use "danguage" as an example to demonstrate my methodology. Finally I will analyze the characteristics of dangerous websites via calsual inference.

To be more specific, I will answer the following questions in this project:

1. How to ascertain whether a website belongs to a specific website category without having a dataset? Not every classification comes with a "ground truth" available for our study. Therefore, I aim to develop a method to determine whether a website falls into a given category in the absence of "ground truth".(i.e. Input: Category defination, website list.  Output: Website which belongs to this category).

    2. Case study: How to identify dangerous websites? I will use "dangerous websites" as an example to demonstrate my methodology. I will develop a method to identify dangerous websites and analyze the characteristics of dangerous websites via calsual inference(**Not sure which method for this part**).

In this projcet I aim to create a list of dangerous websites and conduct a feature analysis on the websites in this list. I want to determine what kinds of websites are more likely to be considered dangerous, what characteristics dangerous websites have, and observe the trackers and advertisements on these websites. I wish to understand how to identify dangerous websites and whether we can predict dangerous websites. I believe that dangerous websites are a collection of certain types of websites. Through this work, we could better identify dangerous websites. Zeng[^1] proposed how to identify a bad advertisement in 2021, which provided ideas for my work. Englehardt and others analyzed the security of popular websites in 2016[^2], but no one has analyzed a large scale of specific types of websites, so my work should be meaningful.

This project's contribution will be:

1. Develop a generalizable novel methodology for identifying website list in a target category.

2. Analyze the characteristics of dangerous websites via calsual inference.





### Process

#### 1.Collect a list of websites and the data we want to use.

Now we use data from [ISCX-URL2016](https://www.unb.ca/cic/datasets/url-2016.html)



According to the reference we can use the following features to identify the category of a website:

| Feature ID | Variable Name                      | Variable Description                                                |
|------------|------------------------------------|---------------------------------------------------------------------|
| 1          | IP_Address_Usage                   | Indicates whether the IP address is used as the URL.               |
| 2          | Long_URL                           | Reflects if the URL is unusually long to hide suspicious parts.    |
| 3          | At_Symbol_In_URL                   | Denotes the presence of '@' symbol in the URL.                     |
| 4          | Domain_Prefix_Suffix               | Presence of prefix or suffix in the domain.                        |
| 5          | Sub_Domain_Count                   | Number of sub-domains and multi sub-domains.                       |
| 6          | Request_URL                        | The URL which is requested.                                        |
| 7          | Anchor_URL                         | The URL of anchor links in the webpage.                            |
| 8          | Abnormal_URL                       | Identifies if the URL structure is abnormal.                       |
| 9          | Links_In_Tags                      | Number of links present within tags (like `<a>`, `<img>` etc.).    |
| 10         | Webpage_Title                      | The title of the webpage.                                          |
| 11         | Meta_Description                   | Description of the webpage provided in meta tags (if available).  |
| 12         | Redirect_Page                      | Indicates whether the webpage redirects to another page.           |
| 13         | Active_Content_Presence            | Presence of active content like JavaScript, Flash.                 |
| 14         | Technology_Stack_Info              | Information on the technology stack used, analyzed from HTTP headers or page content. |
| 15         | HTTP_Header_Info                   | Information provided in the HTTP headers of the webpage.           |
| 16         | Privacy_Policy_Presence            | Indicates the presence of Privacy Policy/Terms of Service through text analysis. |
| 17         | Mobile_Responsiveness              | Indicates whether the webpage is designed responsively for mobile devices. |
| 18         | External_Links_Count               | Number of external links present on the webpage by analyzing `<a>` tags. |


For fetching the features above, we write a class [WebScraper](../codes/01-data-gathering/WebScraper.py) to gethering those data.





